In [1]:
# imports
import os
import sys
import time
import traceback
import threading

from qumulo.rest_client import RestClient

### Recursive directory create function

In [2]:
def create_dir(rc, path):
    path_parts = path.split('/')
    for i in range(2, len(path_parts)+1):
        full_path = '/'.join(path_parts[0:i])
        parent_path = '/'.join(path_parts[0:i-1])
        if parent_path == '':
            parent_path = '/'
        name = path_parts[i-1]
        dir_exists = False
        try:
            rc.fs.get_attr(path=full_path)
            dir_exists = True
        except Exception as e:
            if "Error 404: fs_no_such_entry_error" not in str(e):
                print("Unexpected excpetion: %s" % e)
        if not dir_exists:
            rc.fs.create_directory(name = name, dir_path = parent_path)

### Setup all of the variables

In [3]:
cluster = '<qumulo-cluster>'    # Qumulo cluster hostname or IP where you're setting up users
api_user = '<qumulo-user>'     # Qumulo api user name
api_password = '<qumulo-password>' # Qumulo api password
base_directory = 'users-replicated' # the parent path where the users will be created.
user_name = 'tommy' # the "username"
root_path = '/home/tommy'

# If you're setting up replication, you'll need these variables
replicate_to_cluster = '10.100.1.52' # must be an ip address
replicate_to_api_user = '<qumulo-user>'
replicate_to_api_password = '<qumulo-password>'
replicate_to_directory = 'users-backup'

### Initialize reset clients and set up initial directories

In [9]:
rc = RestClient(cluster, 8000)
rc.login(api_user, api_password)

rc_dest = RestClient(replicate_to_cluster, 8000)
rc_dest.login(replicate_to_api_user, replicate_to_api_password);

source_path = '%s/%s/%s' % (root_path, base_directory,  user_name)
destination_path = '%s/%s/%s' % (root_path, replicate_to_directory,  user_name)
print("     Source path: %s:%s" % (cluster, source_path))
print("Destination path: %s:%s" % (replicate_to_cluster, destination_path))

create_dir(rc, source_path)
create_dir(rc_dest, destination_path)

     Source path: product:/home/tommy/users-replicated/tommy
Destination path: 10.100.1.52:/home/tommy/users-backup/tommy


### Set up the replication relationship

In [11]:
replic_res = None
source_id = rc.fs.get_attr(path=source_path)['id']

try:    
    replic_res = rc.replication.create_source_relationship(source = source_id,
                                        target_path = destination_path,
                                        address = replicate_to_cluster,
                                      )
    print("Replication relationship id: %s" % replic_res['id'])
except:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("!!! Exception: %s" % exc_value)

{u'blackout_window_timezone': u'', u'source_root_id': u'189001158', u'target_address': u'10.100.1.52', u'blackout_windows': [], u'target_port': 3712, u'continuous_replication_enabled': True, u'id': u'ed70a154-3c1b-4ac6-a313-3d6dabb09525'}


### Authorize the replication relationship

In [12]:
replic_auth_res = None
try:    
    replic_auth_res = rc_dest.replication.authorize(relationship_id = replic_res['id'])
    print("Replication relationship status: %s" % replic_auth_res['state'])
except:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("!!! Exception: %s" % exc_value)

Replication relationship status: ESTABLISHED


In [ ]:
# Kick off a replication job.
if replic_res is not None:
    rc.replication.replicate(relationship = replic_res['id'])

### Delete the replication relationship and destination directory

In [ ]:
if replic_res is not None:
    rc.replication.delete_relationship(relationship_id = replic_res['id'])
    rc_dest.fs.delete_tree(path = '/users-backup/%s' % user_name)